In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize 
import io
import os
import re

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
training_data=pd.read_csv('train.csv')
training_data["text"][0]


'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

In [5]:
testing_data=pd.read_csv("test.csv")
testing_data["text"][0]

"@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?"

In [6]:
x=training_data["text"]
type(x)

pandas.core.series.Series

In [7]:
xtest=testing_data["text"]
xtest=np.array(xtest)
xtest=xtest.reshape(len(xtest),1)
xtest.shape

(3660, 1)

In [8]:
y=training_data['airline_sentiment']
x=np.array(x)
y=np.array(y)
x=x.reshape(len(x),1)
y=y.reshape(len(y),1)
x.shape,y.shape

((10980, 1), (10980, 1))

In [9]:
train=np.append(x,y,axis=1)
train.shape

(10980, 2)

In [10]:
test=xtest
test.shape

(3660, 1)

In [11]:
documents = []
c=0
for text,category in train:
    c+=1
    documents.append((word_tokenize(text), category))
print("The End")
documents[0:1]

The End


[(['@',
   'SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative')]

In [12]:
test_documents = []
c=0
for text in test:
    c+=1
    test_documents.append(word_tokenize(str(text)))
print("The End")
test_documents[0:2]

The End


[['[',
  '``',
  '@',
  'AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW',
  '.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads',
  '.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr',
  '.',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'for',
  'AA2450',
  '.',
  'Wat',
  '2',
  'do',
  '?',
  "''",
  ']'],
 ['[',
  "'",
  '@',
  'AmericanAir',
  'after',
  'all',
  ',',
  'the',
  'plane',
  'didn',
  '’',
  't',
  'land',
  'in',
  'identical',
  'or',
  'worse',
  ')',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs',
  '.',
  "'",
  ']']]

In [13]:
import random
random.shuffle(documents)
documents[0:2]

[(['@',
   'united',
   'Hubby',
   'made',
   'it',
   'by',
   'the',
   'skin',
   'of',
   'his',
   'teeth',
   '!',
   ':',
   ')'],
  'positive'),
 (['@',
   'JetBlue',
   'Yep',
   '.',
   '1',
   'hour',
   'at',
   'Logan',
   '.',
   'Gangway',
   'broke',
   'and',
   'cabin',
   'crew',
   'stuck',
   'on',
   'plane',
   '.',
   'They',
   'just',
   'arrived',
   '.',
   'Passengers',
   'request',
   'butt',
   'massages',
   '.'],
  'negative')]

In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [15]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [17]:
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
w = "better"
pos_tag([w])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('better', 'RBR')]

In [18]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [19]:
def clean_review(words):
    output_words = []
    #regex = re.compile('[a-zA-Z0-9_-]+$')
    regex = re.compile('[a-zA-Z_-]+$')
    for w in words:
        if w.lower() not in stops and len(w)>2 and not w.isnumeric() and re.match(regex,w):
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [20]:
documents = [(clean_review(document), category) for document, category in documents]
test_documents = [clean_review(document) for document in test_documents]
training_documents = documents
testing_documents = test_documents

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
categories = [category for document, category in training_documents]


In [22]:
text_documents = [" ".join(document) for document, category in training_documents]
test_text_documents = [" ".join(document) for document in testing_documents]

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
count_vec = CountVectorizer(max_features = 7000)
x_train_features = count_vec.fit_transform(text_documents)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [25]:
x_train_features.shape


(10980, 7000)

In [26]:
regex = re.compile('[a-zA-Z_-]+$')
re.match(regex,'tusar')

<re.Match object; span=(0, 5), match='tusar'>

In [29]:
x_test_features = count_vec.transform(test_text_documents)
x_test_features

<3660x7000 sparse matrix of type '<class 'numpy.int64'>'
	with 31548 stored elements in Compressed Sparse Row format>

In [30]:
xtrain=x_train_features
ytrain=categories
xtest=x_test_features

In [34]:
import numpy as np
from sklearn.svm import SVC
svc = SVC(C=0.2,kernel='linear')
svc.fit(xtrain, ytrain)
#svc.score(xtest, ytest)
svc=svc.predict(xtest)
svc=np.array(svc)
np.savetxt("final_sentiment.csv",svc, fmt='%s',encoding=None)

In [35]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [37]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.83)
clf.fit(xtrain.toarray(), ytrain)
mnb=clf.predict(xtest)
mnb=np.array(mnb)